<a href="https://colab.research.google.com/github/Olesya002/Image_classifier/blob/main/UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gradio

In [ ]:
pip install piqa

In [7]:
import gradio as gr
from joblib import load
import cv2
import torch
import piqa
import numpy as np

model = load('final_model.joblib')
fid_metric = piqa.FID()
class_names = ['document', 'image', 'text-embeded image']

def classifier(image):
    #img_array = imread(image, as_gray=False)
    if len(image.shape) != 3 or image.shape[2] != 3:
      image =  cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    size = (250, 250)
    resized_img = cv2.resize(image, size)
    #bright_img = cv2.add(resized_img, 50)
    img_tensor = torch.tensor(resized_img).permute(2, 0, 1)[None, ...] / 255
    img_feats = fid_metric.features(img_tensor).reshape(-1).reshape(1,-1)
    prediction = model.predict_proba(img_feats)
    index = np.argmax(prediction)
    class_name = class_names[index]
    confidence_score = prediction[0][index]
    return class_name, confidence_score

#define gradio interface and other parameters
app =  gr.Interface(
   fn = classifier,
   inputs = gr.Image(label='Upload an image (jpg, jpeg, bmp):'),
   outputs =  [gr.components.Textbox(label = 'Result:'), gr.components.Textbox(label = 'Confidence score:')],
   allow_flagging=False)

app.launch(share = True)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SVC from version 1.3.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 118MB/s]
/usr/local/lib/python3.10/dist-packages/gradio/interface.py:374: UserWarning: The `allow_flagging` parameter in `Interface` nowtakes a string value ('auto', 'manual', or 'never'), not a boolean. Setting parameter to: 'never'.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f4c10caa7a99caab98.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
